In [1]:
import requests

from tqdm import tqdm

## Fetching a dictionary and preparing a wordlist

The dictionary used here is a standard dictionary of words allowed in English Scrabble.

In [2]:
DICTIONARY_URL = 'https://misc-static.s3.amazonaws.com/words.txt'
response = requests.get(DICTIONARY_URL)
dictionary_words = response.text.split()

def wordcount_report(words):
    print('{wordcount} words in dictionary.'.format(wordcount=len(words)))

wordcount_report(dictionary_words)

276643 words in dictionary.


We eliminate words that are shorter than 4 letters long to match the original NYT rules.

In [3]:
fl_words = [word for word in dictionary_words if len(word) >= 4]
wordcount_report(fl_words)

275178 words in dictionary.


In [4]:
by_length = {}

for word in fl_words:
    length = len(word)
    by_length[length] = by_length.get(length, 0) + 1
    
for key in sorted(by_length.keys()):
    print('{wordcount} words have {lettercount} letters.'.format(wordcount=by_length[key], lettercount=key))

5625 words have 4 letters.
12917 words have 5 letters.
22938 words have 6 letters.
34167 words have 7 letters.
41882 words have 8 letters.
42290 words have 9 letters.
36593 words have 10 letters.
28617 words have 11 letters.
20775 words have 12 letters.
14185 words have 13 letters.
9312 words have 14 letters.
5877 words have 15 letters.


## Building puzzles via a reverse map strategy

All puzzles are key:value pairs where the __key__ is a set of 7 letters that represent all the unique letters in a certain word grouping (letters can be duplicated within a word), and the __value__ is a list of words that can only contain letters from the 7-letters set.

For example, a valid puzzle could be the following.
```
"AEBRTPK": [ "BREAK", "BRAKE", "BRAT", "TRAP"... ]
```

Instead of generating keys and finding words that can be associated with them, we first map all the words in the wordlist based on the set of letters needed to compose them. Letters that share the same set of letters are part of the same puzzle. In this scenario, we use a `frozenset` of letters as a key. At this point, keys are not necessarily 7 items long.

In [5]:
puzzles_rev_map = {}

for word in tqdm(fl_words):
    letter_set = frozenset(word)
    puzzles_rev_map[letter_set] = puzzles_rev_map.get(letter_set, []) + [word]

print('{classcount} possible puzzles'.format(classcount=len(puzzles_rev_map)))

100%|██████████| 275178/275178 [00:01<00:00, 266561.64it/s]

104867 possible puzzles


In [6]:
puzzles_by_length = {}

for puzzle in tqdm(puzzles_rev_map.keys()):
    puzzles_by_length[len(puzzle)] = puzzles_by_length.get(len(puzzle), []) + [puzzle]
    
for key in sorted(list(puzzles_by_length.keys())):
    print('There are {} puzzles with {} letters'.format(len(puzzles_by_length[key]), key))

100%|██████████| 104867/104867 [00:32<00:00, 3226.13it/s]

There are 59 puzzles with 2 letters
There are 680 puzzles with 3 letters
There are 3546 puzzles with 4 letters
There are 8503 puzzles with 5 letters
There are 15026 puzzles with 6 letters
There are 20852 puzzles with 7 letters
There are 21870 puzzles with 8 letters
There are 17419 puzzles with 9 letters
There are 10476 puzzles with 10 letters
There are 4803 puzzles with 11 letters
There are 1399 puzzles with 12 letters
There are 213 puzzles with 13 letters
There are 19 puzzles with 14 letters
There are 2 puzzles with 15 letters


## Combining smaller puzzles with their supersets

To produce a list of 7-letter puzzles, we need to combine lower-tier puzzles with any puzzle that is their superset. For instance, two puzzles `"ABC"` and `"AB"` can contain different words, but all words that belong to `"AB"` are also possible under `"ABC"`. We will then collapse those two together such that `"ABC"` contains all the lower tier words that can be produced from a subset of its letters.

In [7]:
classes = sorted(list(puzzles_by_length.keys()))

for index in range(len(classes)):
    if index >= len(classes)+1 or classes[index] > 7:
        break

    current_class = puzzles_by_length[classes[index]]
    next_class = puzzles_by_length[classes[index+1]]

    for puzzle in tqdm(next_class):
        for subpuzzle in current_class:
            if puzzle.issuperset(subpuzzle):
                puzzles_rev_map[puzzle] = list(set(puzzles_rev_map[puzzle] + puzzles_rev_map[subpuzzle]))

100%|██████████| 21870/21870 [03:01<00:00, 120.74it/s]


We also attempt to eliminate puzzles that have less than 50 possible words or more than 100 to avoid having puzzles where the possible answer count is too large.

In [8]:
seven_letter_puzzles = puzzles_by_length[7]
trimmed_puzzles = [puzzle for puzzle in seven_letter_puzzles if len(puzzles_rev_map[puzzle]) <= 100 and len(puzzles_rev_map[puzzle]) > 50]

In [9]:
puzzle_set = { ''.join(list(puzzle)):puzzles_rev_map[puzzle] for puzzle in trimmed_puzzles }
len(puzzle_set)

1195

## Export

Finally, we export the puzzle list (i.e. the list of key:value pairs discussed earlier) as JSON so that we can pull it and use it as a source of truth for puzzles.

At this point, it contains more than 

In [10]:
import json


encoded = json.dumps(puzzle_set).encode('utf-8')
print(len(encoded))

with open('test.txt', 'w') as outfile:
    outfile.write(encoded.decode('utf-8'))

865770
